In [52]:
from bs4 import BeautifulSoup
import requests

In [53]:
req = requests.get("https://www.mathjobs.org/jobs?joblist-0-0----rss--")

In [54]:
req.ok

True

In [15]:
!pip install xml

ERROR: Could not find a version that satisfies the requirement xml (from versions: none)
ERROR: No matching distribution found for xml


In [55]:
soup = BeautifulSoup(req.text, 'xml')

In [56]:
items = soup.find_all("item")

In [57]:
len(items)

477

In [58]:
req2 = requests.get(items[5].find("link").text)

In [59]:
req2.ok

True

In [60]:
#import pprint
#pprint.pprint(req2.text)

In [61]:
soup2 = BeautifulSoup(req2.text, 'html5')

In [62]:
soup2.find_all("b")[2]

<b>Position Type:</b>

In [50]:
soup2.find_all("b")[2].next_sibling

' Postdoctoral'

In [63]:
from collections import Counter

In [66]:
ca = Counter()
for res in soup2.find_all("b"):
    ca.update(res[2].text)

KeyError: 2

In [68]:
!pip install requests_futures

In [70]:
from requests_futures.sessions import FuturesSession

results = []
session = FuturesSession(max_workers=5) # max_workers should not be too high so you don't make too many requests per second and get blocked

fails = []
for i,future in enumerate([session.get(item.find("link").text) for item in items]):
    try:
        results.append(future.result())
    except:
        fails.append(i)

In [79]:
position_labels = []
positions = []
new_fails = []
for i, res in enumerate(results):
    if res.ok:  
        soup2 = BeautifulSoup(res.text, 'html5')
        position_labels.append(soup2.find_all("b")[2].text)
        positions.append(soup2.find_all("b")[2].next_sibling)
    else:
        new_fails.append(i)

In [81]:
pos_type = Counter(position_labels)
pos_type

Counter({'Position Type:': 417,
         'Fellowship Type:': 26,
         'Position Location:': 26,
         'Scholarship Location:': 1,
         'Fellowship Location:': 2,
         'Application Type:': 2,
         'Program Type:': 3})

In [82]:
positions_cn = Counter(positions)
positions_cn

Counter({' Tenured/Tenure-track faculty': 213,
         ' Postdoctoral': 122,
         ' Non tenure-track faculty': 73,
         ' Fellowship or award': 9,
         ' Other': 12,
         ' Government or industry': 15,
         ' St. Thomas, St. Thomas 00802, Virgin Islands (U.S.) [': 1,
         ' London, London, United Kingdom [': 1,
         ' London, London E1 4NS, United Kingdom [': 1,
         ' Chongqing, Chongqing, China [': 4,
         ' Fairfax, Virginia 22030, United States [': 1,
         ' Stony Brook, New York 11794-3651, United States [': 1,
         ' Logan, Utah 84322, United States [': 1,
         ' Buffalo, New York 14260, United States [': 1,
         ' New York, New York 10027, United States [': 1,
         ' Boston, Massachusetts 02215, United States [': 1,
         ' Notre Dame, Indiana 46556, United States': 1,
         ' Tuscaloosa, Alabama 35487, United States [': 1,
         ' Thuwal, Jeddah 23955-6900, Saudi Arabia [': 2,
         ' Toronto, Ontario M5T3J1, 

In [85]:
213+122+73+15

423